In [7]:
import requests
import json
import pandas as pd
from pathlib import Path


out_dir = Path("data/spacex")
out_dir.mkdir(parents=True, exist_ok=True)


endpoints = {
    "rockets": "https://api.spacexdata.com/v4/rockets",
    "launches": "https://api.spacexdata.com/v4/launches",
    "capsules": "https://api.spacexdata.com/v4/capsules",
}


for name, url in endpoints.items():
    print(f"Baixando {name}...")
    resp = requests.get(url)
    data = resp.json()


    with open(out_dir / f"{name}.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

print("Arquivos salvos em data/spacex/")

dfs = {}
for name in endpoints.keys():
    print(f"Carregando {name}...")
    df = pd.read_json(out_dir / f"{name}.json")
    dfs[name] = df
rockets_df = dfs["rockets"][["id", "name", "first_flight", "country", "company", "stages", "boosters", "cost_per_launch", "success_rate_pct", "height", "mass"]]
launches_df = dfs["launches"][["id", "name", "date_utc", "success", "rocket", "details"]]
capsules_df = dfs["capsules"][["id", "type", "status", "serial", "reuse_count", "water_landings", "land_landings"]]

print("\n=== Rockets ===")
display(rockets_df.head())

print("\n=== Launches ===")
display(launches_df.head())

print("\n=== Capsules ===")
display(capsules_df.head())


Baixando rockets...
Baixando launches...
Baixando capsules...
Arquivos salvos em data/spacex/
Carregando rockets...
Carregando launches...
Carregando capsules...

=== Rockets ===


,id,name,first_flight,country,company,stages,boosters,cost_per_launch,success_rate_pct,height,mass
0,5e9d0d95eda69955f709d1eb,Falcon 1,2006-03-24,Republic of the Marshall Islands,SpaceX,2,0,6700000,40,"{'meters': 22.25, 'feet': 73}","{'kg': 30146, 'lb': 66460}"
1,5e9d0d95eda69973a809d1ec,Falcon 9,2010-06-04,United States,SpaceX,2,0,50000000,98,"{'meters': 70, 'feet': 229.6}","{'kg': 549054, 'lb': 1207920}"
2,5e9d0d95eda69974db09d1ed,Falcon Heavy,2018-02-06,United States,SpaceX,2,2,90000000,100,"{'meters': 70, 'feet': 229.6}","{'kg': 1420788, 'lb': 3125735}"
3,5e9d0d96eda699382d09d1ee,Starship,2021-12-01,United States,SpaceX,2,0,7000000,0,"{'meters': 118, 'feet': 387}","{'kg': 1335000, 'lb': 2943000}"



=== Launches ===


,id,name,date_utc,success,rocket,details
0,5eb87cd9ffd86e000604b32a,FalconSat,2006-03-24T22:30:00.000Z,0.0,5e9d0d95eda69955f709d1eb,Engine failure at 33 seconds and loss of vehicle
1,5eb87cdaffd86e000604b32b,DemoSat,2007-03-21T01:10:00.000Z,0.0,5e9d0d95eda69955f709d1eb,Successful first stage burn and transition to ...
2,5eb87cdbffd86e000604b32c,Trailblazer,2008-08-03T03:34:00.000Z,0.0,5e9d0d95eda69955f709d1eb,Residual stage 1 thrust led to collision betwe...
3,5eb87cdbffd86e000604b32d,RatSat,2008-09-28T23:15:00.000Z,1.0,5e9d0d95eda69955f709d1eb,Ratsat was carried to orbit on the first succe...
4,5eb87cdcffd86e000604b32e,RazakSat,2009-07-13T03:35:00.000Z,1.0,5e9d0d95eda69955f709d1eb,None



=== Capsules ===


,id,type,status,serial,reuse_count,water_landings,land_landings
0,5e9e2c5bf35918ed873b2664,Dragon 1.0,retired,C101,0,1,0
1,5e9e2c5bf3591882af3b2665,Dragon 1.0,retired,C102,0,1,0
2,5e9e2c5bf3591835983b2666,Dragon 1.0,unknown,C103,0,1,0
3,5e9e2c5bf359189ef23b2667,Dragon 1.0,unknown,C104,0,1,0
4,5e9e2c5bf3591859a63b2668,Dragon 1.1,unknown,C105,0,1,0


In [8]:
# Relações entre os DataFrames
rockets_df = dfs["rockets"]
launches_df = dfs["launches"]
capsules_df = dfs["capsules"]

# 1. Relação Launches -> Rockets
print("\nValidação Launches -> Rockets")
rocket_ids_in_launches = set(launches_df["rocket"].dropna().unique())
rocket_ids_master = set(rockets_df["id"].unique())
print("IDs de foguetes usados em launches que não estão em rockets:")
print(rocket_ids_in_launches - rocket_ids_master)

# 2. Relação Launches -> Capsules
print("\nValidação Launches -> Capsules")
all_capsule_ids_in_launches = set([c for sublist in launches_df["capsules"] if isinstance(sublist, list) for c in sublist])
capsule_ids_master = set(capsules_df["id"].unique())
print("IDs de cápsulas usados em launches que não estão em capsules:")
print(all_capsule_ids_in_launches - capsule_ids_master)



Validação Launches -> Rockets
IDs de foguetes usados em launches que não estão em rockets:
set()

Validação Launches -> Capsules
IDs de cápsulas usados em launches que não estão em capsules:
set()
